In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import copy

In [2]:
frame_num = 1
file_path = r'C:\Users\zhang\Desktop\share\18knpt_1000p_out.xyz'
new_name = r'C:\Users\zhang\Desktop\18knpt_ana.lammpstrj'

In [3]:
#读取文本函数，返回直接d选帧
def get_data(path,frame_num):
    with open(path) as f:
        str_data = f.read()
    data = str_data.split()
    ###选帧#########
    #此处写个程序选帧
    atoms_num = int(data[0])
    frame_len = atoms_num*4+2
    data_len =len(data)
    max_frame = data_len // frame_len
    
    temp = []
    temp = data[frame_len*frame_num:frame_len*(frame_num+1)]

    #暂时先自行处理了表头
    temp = temp[2:]
    frame_list = []
    for i in range(atoms_num):
        frame_list.append([temp[i*4+0],float(temp[i*4+1]),float(temp[i*4+2]),float(temp[i*4+3])])
    
    return frame_list

In [4]:
# 定义距离函数
def cal_distance(x1,y1,z1,x2,y2,z2):
    dis = ((x1-x2)**2+(y1-y2)**2+(z1-z2)**2)**(1/2)
    if dis<1:
        return 10
    return dis

# 主函数

In [5]:
start_frame = 1
end_frame = 200
step_frame = 2

#看文件是否存在 存在也删了 都要重建
if not os.path.exists(new_name):
    os.system(r'touch {}'.format(new_name))
else:
    os.remove(new_name)
    os.system(r'touch {}'.format(new_name))
    
for frame in range(start_frame,end_frame,step_frame):
    frame_count = int((end_frame-start_frame)/step_frame)
# 获取数据
    data = get_data(file_path,frame).copy()
# 处理数据准备
    num_cs = 0
    max_xyz = 0
    dic_ele = {}
    ba_list = []
    all_atoms = 0
    for i in data:
        all_atoms+=1
        if i[1]>max_xyz:
            max_xyz = i[1]
        if i[2]>max_xyz:
            max_xyz = i[2]
        if i[3]>max_xyz:
            max_xyz = i[3]
        
        # 统计元素种类及个数
        if i[0] not in dic_ele:
            dic_ele[i[0]] = 0
        else:
            dic_ele[i[0]]+=1
        # 得到ba列表
        if i[0]=='Ba':
            ba_list.append(i.copy())
    
    
#  开始处理数据
    for ba_i in ba_list:
        count_ba_ba = 0
        count_al_si = 0
        count_o = 0
        count_f = 0
    
        for j in data:
            # 如果是同一原子则跳过 这个在计算距离的时候判断就行了  不然计算太多遍了
            # if (i[1]-j[1]<1e-4 and i[2]-j[2]<1e-4 and i[3]-j[3]
            if j[0]=='Ba':
                if cal_distance(ba_i[1],ba_i[2],ba_i[3],j[1],j[2],j[3])< 6.0:
                    count_ba_ba+=1
            if j[0]=='Al' or j[0]=='Si':
                if cal_distance(ba_i[1],ba_i[2],ba_i[3],j[1],j[2],j[3])< 6.0:
                    count_al_si+=1
            if j[0]=='O' :
                if cal_distance(ba_i[1],ba_i[2],ba_i[3],j[1],j[2],j[3])< 3.5:
                    count_o+=1
            if j[0]=='F' :
                if cal_distance(ba_i[1],ba_i[2],ba_i[3],j[1],j[2],j[3])< 3.5:
                    count_f+=1

        #综合判断
        if(count_al_si<=count_ba_ba and count_o <= count_f):
            ba_i[0]='Cs'
    
# 储存回文本
#先把文件头写上去
    with open(new_name,'a') as file:
# 文件头
        file.write("ITEM: TIMESTEP\n")
        #  帧数
        file.write(str(frame_count) +'\n')
        file.write("ITEM: NUMBER OF ATOMS\n")
        # 原子数
        file.write(str(all_atoms) +'\n')
        file.write("ITEM: BOX BOUNDS xy xz yz pp pp pp\n\
0.0    100.0  0.0 \n\
0.0    100.0  0.0 \n\
0.0    100.0  0.0 \n\
ITEM: ATOMS type x y z vx vy vz\n")    
# 写回数据
        for i_d in data:
            if i_d[0]!='Ba':
                file.write(str(i_d[0]) + '  ' + str(i_d[1]) +'  '+ str(i_d[2]) + '  ' + str(i_d[3]) +  ' 0 0 0 '+ '\n')
            if i_d[0]=='Cs':
                print('nonono')
        
        for j in ba_list:
            if j[0]=='Ba':
                file.write(str(j[0]) + '  ' + str(j[1]) +'  '+ str(j[2]) + '  ' + str(j[3])+ ' 0 0 0 '+ '\n')
            elif j[0]=='Cs':
                num_cs+=1 
                file.write('Ba' +  '  ' + str(j[1]) +'  '+ str(j[2]) +'  '+ str(j[3])+ ' 1 1 1 ' +  '\n')
            else:
                print('no')
    # 输出相关信息
    print('帧数：',frame,'  Cs原子数： ',num_cs)

帧数： 1   Cs原子数：  33
帧数： 3   Cs原子数：  35
帧数： 5   Cs原子数：  36
帧数： 7   Cs原子数：  41
帧数： 9   Cs原子数：  44
帧数： 11   Cs原子数：  35
帧数： 13   Cs原子数：  48
帧数： 15   Cs原子数：  68
帧数： 17   Cs原子数：  51
帧数： 19   Cs原子数：  56
帧数： 21   Cs原子数：  59
帧数： 23   Cs原子数：  65
帧数： 25   Cs原子数：  61
帧数： 27   Cs原子数：  81
帧数： 29   Cs原子数：  65
帧数： 31   Cs原子数：  54
帧数： 33   Cs原子数：  81
帧数： 35   Cs原子数：  74
帧数： 37   Cs原子数：  78
帧数： 39   Cs原子数：  67
帧数： 41   Cs原子数：  78
帧数： 43   Cs原子数：  98
帧数： 45   Cs原子数：  92
帧数： 47   Cs原子数：  104
帧数： 49   Cs原子数：  100
帧数： 51   Cs原子数：  112
帧数： 53   Cs原子数：  116
帧数： 55   Cs原子数：  109
帧数： 57   Cs原子数：  107
帧数： 59   Cs原子数：  118
帧数： 61   Cs原子数：  117
帧数： 63   Cs原子数：  119
帧数： 65   Cs原子数：  126
帧数： 67   Cs原子数：  129
帧数： 69   Cs原子数：  125
帧数： 71   Cs原子数：  113
帧数： 73   Cs原子数：  120
帧数： 75   Cs原子数：  120
帧数： 77   Cs原子数：  108
帧数： 79   Cs原子数：  109
帧数： 81   Cs原子数：  119
帧数： 83   Cs原子数：  123
帧数： 85   Cs原子数：  116
帧数： 87   Cs原子数：  132
帧数： 89   Cs原子数：  121
帧数： 91   Cs原子数：  117
帧数： 93   Cs原子数：  113
帧数： 95   Cs原子数：  118
帧数： 97   Cs原子数：  129

In [6]:
print('ok')

ok


# 先设置最大值xyz的最大值 由于其为npt，所以其xyz边界是变化的，那么最大的就动态获取
max_xyz = 0
dic_ele = {}
ba_list = []
all_atoms = 0
for i in data:
    all_atoms+=1
    if i[1]>max_xyz:
        max_xyz = i[1]
    if i[2]>max_xyz:
        max_xyz = i[2]
    if i[3]>max_xyz:
        max_xyz = i[3]
        
    # 统计元素种类及个数
    if i[0] not in dic_ele:
        dic_ele[i[0]] = 0
    else:
        dic_ele[i[0]]+=1
    # 得到ba列表
    if i[0]=='Ba':
        ba_list.append(i)
print(max_xyz)    
print(dic_ele)